## Import Some Packages

In [1]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader,random_split

import math
# For data preprocess
import numpy as np
import csv
import os
from sklearn.preprocessing import MinMaxScaler
# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# For txt
import sys
time_stand = 2024250000000.000
import matplotlib
# Non-interactive backend, you can't call plt.show() to see the figure interactively
# matplotlib.use('Agg') must be placed before import matplotlib.pyplot
matplotlib.use('Agg') 
import matplotlib.pyplot as plt

## Data Process

In [2]:
path_SL = 'Data/statlink01_orbit.txt'
path_Lunar = 'Data/Lunar_Vector_J2000.txt'
path_Sun = 'Data/Sun_Vector_J2000.txt'

def datacatcher(path,idx_start,idx_end,line_pass):
    f = open(path , encoding='utf-8')
    line = f.readline()
    list = []
    count = 0
    while line:
        if(count%line_pass == 0):
            a = line.split(" ")            #将数据以空格的方式分隔开
            b = a[idx_start:idx_end]              #这就是选择前四行保存下来（如果想保存第2，3行就写成b = a[1,3]）即可
            list.append(b)
            # list.append('\n')
        line = f.readline()
        count = count+1
    f.close()
    data_array=np.array(list)
    return data_array.astype(float)
data_SL_raw = datacatcher(path_SL,0,4,4)
data_Lunar = datacatcher(path_Lunar,4,7,1)
data_Sun = datacatcher(path_Sun,4,7,1)
data_time = data_SL_raw[:,:1]-time_stand
data_SL = data_SL_raw[:,1:]# 除去时间特征
slide_step = 1
# Dataset_raw = np.hstack((data_time, data_Sun,data_Lunar, data_SL))#
Dataset_raw = np.hstack((data_SL[:-1*slide_step,:],data_SL[1*slide_step:,:]))#
print(Dataset_raw.shape)



(4320, 6)


## Set Random Seed

In [3]:
myseed = 5201314##42069
def same_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
same_seed(myseed)

## Some Utilities

In [4]:
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 300,               # mini-batch size for dataloader
    'optimizer': 'Adam',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        'lr': 1e-4,                 # learning rate
        # 'momentum': 0.9              # momentum
        'weight_decay': 1e-5        # weight_decay
    },
    'early_stop': 1000,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pth' , # your model will be saved here
    'fig_path': 'fig/',
    'fig_type': 'png',
    'window_len':100*2,                   # T = 100
    'window_step':1,
    'future':100
}
def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def train_valid_split(data_set,valid_ratio,seed):
    # data_set = data_set[:2000]  
    # test_data = data_set[1400:]   #600
    # train_data = data_set[:1200] #1200
    # valid_data = data_set[1200:1400] #200
    test_data = data_set[3000:]   # 1320
    train_data = data_set[:2600] # 2600
    valid_data = data_set[2600:3000] # 400
    print(f'size of test_data is {test_data.shape}')
    print(f'size of train_data is {train_data.shape}')
    print(f'size of valid_data is {valid_data.shape}')
    return np.array(train_data),np.array(valid_data),np.array(test_data)


In [6]:
# 对训练集和测试集进行归一化
m_x = MinMaxScaler()
m_y = MinMaxScaler()
m_z = MinMaxScaler()

# 前三列为Input,归一化就好，不用保存参数
Dataset_raw[:,0] = m_x.fit_transform(Dataset_raw[:,0].reshape(-1, 1)).flatten() #注意fit_transform() 和 transform()的区别
Dataset_raw[:,1] = m_y.fit_transform(Dataset_raw[:,1].reshape(-1, 1)).flatten() #注意fit_transform() 和 transform()的区别
Dataset_raw[:,2] = m_z.fit_transform(Dataset_raw[:,2].reshape(-1, 1)).flatten() #注意fit_transform() 和 transform()的区别
# 后三列为Output,归一化还需保存参数，反归一化再用
Dataset_raw[:,3] = m_x.fit_transform(Dataset_raw[:,3].reshape(-1, 1)).flatten() #注意fit_transform() 和 transform()的区别
Dataset_raw[:,4] = m_y.fit_transform(Dataset_raw[:,4].reshape(-1, 1)).flatten() #注意fit_transform() 和 transform()的区别
Dataset_raw[:,5] = m_z.fit_transform(Dataset_raw[:,5].reshape(-1, 1)).flatten() #注意fit_transform() 和 transform()的区别
# 分解数据
valid_ratio = 0.1
train_data, valid_data, test_data = train_valid_split(Dataset_raw,valid_ratio,myseed)
train_data = train_data.astype(float)
valid_data = valid_data.astype(float)
test_data = test_data.astype(float)


size of test_data is (1320, 6)
size of train_data is (2600, 6)
size of valid_data is (400, 6)


##  Set Dataset

In [7]:
class OrbitDataset(Dataset):
    # input:数据路径；模式（默认为train）；特征选择
    def __init__(self,
                 data,
                 mode='train'):
        # Determine the mode
        self.mode = mode
        # 检查数据类型
        data = np.array(data)  # Convert to numpy array if not already
        print(data.dtype)  # 查看数据类型
        if mode == 'test':
            # Testing data
            # data: 321 x 10 
            data = data[:, 0]
            self.data = torch.FloatTensor(np.array(data)).unsqueeze(1)# 转化成张量
        else:
            target = data[:, 0+3]
            data = data[:, 0]
            # target = data[:, -3:]
            # data = data[:, :3]
            
            # Convert data into PyTorch tensors
            self.data = torch.FloatTensor(np.array(data)).unsqueeze(1)
            self.target = torch.FloatTensor(np.array(target)).unsqueeze(1)
        # self.data[:, 0] = self.data[:, 0]
        # 数据归一化
        # self.data[:, 1:] = \
        #     (self.data[:, 1:] - self.data[:, 1:].mean(dim=0, keepdim=True)) \
        #     / self.data[:, 1:].std(dim=0, keepdim=True)

        # self.dim = self.data.shape[1]# 选取的是第二维度
        self.dim = 1# 选取的是第二维度
        print('Finished reading the {} set of Orbit Dataset ({} samples found, each dim = {})'
              .format(mode, len(self.data), self.dim))

    def __getitem__(self, index):
        # Returns one sample at a time
        if self.mode in ['train', 'dev']:
            # For training
            return self.data[index], self.target[index]
        else:
            # For testing (no target)
            return self.data[index]

    def __len__(self):
        # Returns the size of the dataset
        return len(self.data)

## Set DataLoader

In [8]:
def prep_dataloader(data, mode, congfig, n_jobs=0):
    ''' Generates a dataset, then is put into a dataloader. '''
    dataset = OrbitDataset(data, mode=mode)  # Construct dataset
    batch_x = list()
    batch_y = list()
    window_len = congfig['window_len']
    for end in range(len(dataset.data) - window_len, -1, -1*congfig['window_step']):
        batch_x.append(dataset.data[end:end+window_len])
        if mode in ('train', 'dev'):
            batch_y.append(dataset.target[end:end+window_len])
        else:
            batch_y = batch_x

    # batch_x的shape是(-, window_len, 1) 

    from torch.nn.utils.rnn import pad_sequence
    batch_x = pad_sequence(batch_x)
    batch_y = pad_sequence(batch_y)
 
    # batch_x的shape是(window_len, -, 1)   
    batch_x,batch_y = batch_x.permute(1, 0, 2),batch_y.permute(1, 0, 2)
    # print(f'shape of batch_x:{batch_x.shape}')
    # print(f'shape of batch_y:{batch_y.shape}')
    # return batch_x.permute(1, 0, 2),batch_y.permute(1, 0, 2)
    return batch_x.squeeze(2),batch_y.squeeze(2)


## Set Model

In [9]:
class LSTM_Model(nn.Module):
    def __init__(self):
        super(LSTM_Model, self).__init__()
        # define the dim
        self.input_dim = 1
        self.hid_dim = 51
        self.output_dim = 1
        # set the model parts
        self.lstm1 = nn.LSTMCell(self.input_dim, self.hid_dim)
        self.lstm2 = nn.LSTMCell(self.hid_dim, self.hid_dim)
        self.FC = nn.Sequential(
            nn.Linear(self.hid_dim, 16),
            nn.Sigmoid(),
            nn.Linear(16, 4),
            nn.Sigmoid(),
            nn.Linear(4, self.output_dim),
        )  # FC
        self.criterion = nn.MSELoss(reduction='mean')
    def forward(self, input, future = 0):
        outputs = []
        # c:记忆元里面存放的值
        # h:前一个时间点的输出
        h_t = torch.zeros(input.size(0), self.hid_dim , dtype=torch.float)
        c_t = torch.zeros(input.size(0), self.hid_dim , dtype=torch.float)
        h_t2 = torch.zeros(input.size(0), self.hid_dim, dtype=torch.float)
        c_t2 = torch.zeros(input.size(0), self.hid_dim, dtype=torch.float)

        h_t = h_t.to(device)
        c_t = c_t.to(device)
        h_t2 = h_t2.to(device)
        c_t2 = c_t2.to(device)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.FC(h_t2)  # output.shape:[batch,1]
            outputs += [output] # outputs.shape:[[batch,1],...[batch,1]], list composed of n [batch,1],
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t)) 
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.FC(h_t2) # output.shape:[batch,1]
            outputs += [output]  # outputs.shape:[[batch,1],...[batch,1]], list composed of n [batch,1],
        outputs = torch.stack(outputs, 1).squeeze(2) # shape after stack:[batch, n, 1], shape after squeeze: [batch,n]
        return outputs
    def cal_loss(self, pred, target):
        return self.criterion(pred, target)
    def r2_loss(self, pred, target):
        target_mean = torch.mean(target)
        ss_tot = torch.sum((target - target_mean) ** 2)
        ss_res = torch.sum((target - pred) ** 2)
        r2 = 1 - ss_res / ss_tot
        return r2

## --Train/Dev/Test

In [10]:
# Training
def train(tr_set_x,tr_set_y, dv_set_x,dv_set_y,  model, config, device):
    n_epochs = config['n_epochs']  # Maximum number of epochs
    optimizer = torch.optim.Adam(model.parameters(), lr=config['optim_hparas']['lr'], weight_decay=config['optim_hparas']['weight_decay'])
    min_mse = math.inf # 初始设置为无穷大math.inf也行
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0

    while epoch < n_epochs:
        model.train()                           # set model to training mode
        x,y = tr_set_x,tr_set_y
        # y = y.squeeze(2)
        optimizer.zero_grad()               # set gradient to zero
        x, y= x.to(device), y.to(device)    # move data to device (cpu/cuda)
        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                     torch.zeros(1, 1, model.hidden_layer_size))
        pred = model(x)                     # forward pass (compute output)
        # pred, hidden_prev = model(x, hidden_prev)
        # hidden_prev = hidden_prev.detach()
        # print(f'shape of pred:{pred.shape}')
        # print(f'shape of y:{y.shape}')
        train_r2score = model.r2_loss(pred, y)
        mse_loss = model.cal_loss(pred, y)  # compute loss
        mse_loss.backward()                 # compute gradient (backpropagation)
        optimizer.step()                    # update model with optimizer
        loss_record['train'].append(mse_loss.detach().cpu().item())# 记录train的Loss,放在字典里面
        mean_train_loss = sum(loss_record['train'])/len(loss_record['train'])# 记录训练的loss
        # After each epoch, test your model on the validation (development) set.
        dev_mse,dev_r2score = dev(dv_set_x,dv_set_y, model, device)
        if dev_mse < min_mse:
            # Save model if your model improved
            min_mse = dev_mse
            print('Saving model (Epoch = {:4d}/{:4d}, train_loss = {:.4f}, dev_loss = {:.4f})\n train_r2score = {:.4f},dev_r2score={:.4f}'
                .format(epoch + 1,config['n_epochs'], mean_train_loss, min_mse,train_r2score,dev_r2score))
            torch.save(model.state_dict(), config['save_path'])  # Save model to specified path
            early_stop_cnt = 0
        else:
            # 如果模型没有优化，就记录加一，否则清零
            early_stop_cnt += 1

        epoch += 1
        torch.cuda.empty_cache()
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
            break

    print('Finished training after {} epochs'.format(epoch))
    return pred,min_mse, loss_record

# Validation
def dev(dv_set_x,dv_set_y, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    # for x, y in zip(dv_set_x,dv_set_y):                         # iterate through the dataloader
    x, y = dv_set_x,dv_set_y
    x, y = x.to(device), y.to(device)      # move data to device (cpu/cuda)
    with torch.no_grad():                   # disable gradient calculation
        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                     torch.zeros(1, 1, model.hidden_layer_size))
        pred = model(x)
        dev_r2score = model.r2_loss(pred, y)
        mse_loss = model.cal_loss(pred, y)  # compute loss
    total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set_x)              # compute averaged loss

    return total_loss,dev_r2score
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

## Load data and model

In [11]:
tr_set_x,tr_set_y = prep_dataloader(train_data, 'train', config)
dv_set_x,dv_set_y = prep_dataloader(valid_data, 'dev', config)
tt_set_x,tt_set_y = prep_dataloader(test_data, 'test', config)
# print(tr_set.dataset.dim)
device = get_device()                 # get the current available device ('cpu' or 'cuda')
print(device)
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
model = LSTM_Model().float().to(device)  # Construct model and move to device




float64
Finished reading the train set of Orbit Dataset (2600 samples found, each dim = 1)
float64
Finished reading the dev set of Orbit Dataset (400 samples found, each dim = 1)
float64
Finished reading the test set of Orbit Dataset (1320 samples found, each dim = 1)
cuda


## StartTrain

In [12]:
train(tr_set_x,tr_set_y, dv_set_x,dv_set_y,  model, config, device)

d:\PythonEnviroment\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

## StartPredict

In [199]:
# del model
# # 删除当前的 model 对象。这样做是为了释放内存
# model = LSTM_Model().to(device)
# ckpt = torch.load(config['save_path'], map_location='cpu')  # Load your best model
# model.load_state_dict(ckpt)
input = tt_set_x[0,:].to(device)
print(input.shape)
input = input.unsqueeze(0)

# begin to predict, no need to track gradient here
with torch.no_grad():
    future = config['future']
    preds = model(input, future=future)
    preds = preds.detach().cpu()
    preds = preds.numpy()
preds = np.array(preds)
print(f'the size of input is {input.size()}')
# print(f'the size of input is {preds.size()}')
# 反归一化
preds = m_x.inverse_transform(preds[0,:].reshape(-1, 1)).flatten()
save_pred(preds, 'pred.csv')         # save prediction file to pred.csv

torch.Size([200])
the size of input is torch.Size([1, 200])
Saving results to pred.csv


## Prediction Plotting

In [200]:
from datetime import datetime
# draw the result
# xx = m_x.inverse_transform(tt_set_x[:1,:].permute(1, 0).cpu().numpy()[:config['window_len'],0].reshape(-1, 1)).flatten() #反归一化
xx = m_x.inverse_transform(Dataset_raw[3000:3300,0].reshape(-1, 1)).flatten()
error = xx - preds
print(preds.shape)
print(xx.shape)
print(tt_set_x.shape)


# 直接指定保存的文件夹路径
folder = config['fig_path']  # 请替换为你的文件夹路径
fig_type = config['fig_type']
# draw predict
plt.figure(figsize=(30,10))
plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
plt.xlabel('time', fontsize=24)
plt.ylabel('prediction', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
def draw(yi, color,label):
    plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 3.0, label=label)
    plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 3.0)
draw(preds, 'r','pred')
# plt.plot(np.array(list(range(0,len(xx)))),xx, 'g', linewidth = 2.0)
draw(xx, 'b','true')
plt.legend(fontsize=24)
plt.grid(True)

pdf_filename = f"{folder}predict_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.{fig_type}"
plt.savefig(pdf_filename)
plt.close()
# draw err
plt.figure(figsize=(30,10))
plt.title('error\n(Dashlines are predicted values)', fontsize=30)
plt.xlabel('time', fontsize=24)
plt.ylabel('error', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
def draw(yi, color,label):
    plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 3.0, label=label)
    plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 3.0)
draw(error, 'r','pred')
plt.legend(fontsize=24)
plt.grid(True)

pdf_filename = f"{folder}error_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.{fig_type}"
plt.savefig(pdf_filename)
plt.close()

(300,)
(300,)
torch.Size([1121, 200])


## Plotting

In [201]:
import matplotlib.pyplot as plt
plotdata_pred = preds
x = plotdata_pred
xx = m_x.inverse_transform(tt_set_x.cpu().numpy()[:config['window_len'],0].reshape(-1, 1)).flatten() #反归一化
# xx = tt_set_x.numpy()[:config['window_len'],0]
t = np.array(list(range(0,len(x))))

print(x.size)
print(xx.size)
print(t.size)

# 创建一个散点图
plt.figure(figsize=(10, 6))
# plt.scatter(t,xx,label='true')
plt.scatter(t,x,label='pred')
plt.legend()  # 显示图例
plt.xlabel('Time')
plt.ylabel('Pos')
plt.grid(True)
plt.savefig('plot.png')  # 保存为文件


300
200
300
